Kiểm tra tổng thể csv (Minh truc meeyland)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def inspect_csv(df):
    print("===== CSV OVERVIEW =====")
    print(f"Rows    : {df.shape[0]}")
    print(f"Columns : {df.shape[1]}")
    print("\n--- Data types ---")
    print(df.dtypes)
    print("\n--- Memory usage ---")
    print(round(df.memory_usage(deep=True).sum() / 1024**2, 2), "MB")


In [3]:
df_raw = pd.read_csv("../data/raw/meeyland_hcm_desc_final.csv")
inspect_csv(df_raw)


===== CSV OVERVIEW =====
Rows    : 1488
Columns : 13

--- Data types ---
id                 int64
Page               int64
Title             object
Price_Raw         object
Price_Billion    float64
Area_m2          float64
District          object
Address           object
Bedrooms         float64
Toilets          float64
Post_Time         object
Link              object
Description       object
dtype: object

--- Memory usage ---
2.96 MB


Hàm kiểm tra & THÔNG BÁO số ô thiếu dữ liệu theo từng cột

In [4]:
def report_missing_values(df):
    print("===== MISSING VALUE REPORT =====")
    missing = df.isna().sum()
    percent = (missing / len(df)) * 100
    report = pd.DataFrame({
        "Missing_Count": missing,
        "Percent_Missing (%)": percent.round(2)
    }).sort_values(by="Missing_Count", ascending=False)
    print(report)
    return report


In [5]:
report_missing_values(df_raw)

===== MISSING VALUE REPORT =====
               Missing_Count  Percent_Missing (%)
Toilets                  755                50.74
Bedrooms                 668                44.89
Price_Billion             25                 1.68
Price_Raw                 25                 1.68
Area_m2                    2                 0.13
Post_Time                  1                 0.07
District                   1                 0.07
Title                      1                 0.07
id                         0                 0.00
Page                       0                 0.00
Address                    0                 0.00
Link                       0                 0.00
Description                0                 0.00


,Missing_Count,Percent_Missing (%)
Toilets,755,50.74
Bedrooms,668,44.89
Price_Billion,25,1.68
Price_Raw,25,1.68
Area_m2,2,0.13
Post_Time,1,0.07
District,1,0.07
Title,1,0.07
id,0,0.00
Page,0,0.00


Hàm chuyển đổi Price

In [6]:
def parse_price_raw(price_raw):
    if pd.isna(price_raw):
        return np.nan
    text = str(price_raw).lower().strip()

    match = re.search(r'([\d\.,]+)\s*(tỷ|ty)', text)
    if match:
        return float(match.group(1).replace(',', '.'))

    match = re.search(r'([\d\.,]+)\s*(triệu|trieu)', text)
    if match:
        return float(match.group(1).replace(',', '.')) / 1000

    return np.nan


In [7]:
def clean_price(df):
    print("===== PRICE CLEANING =====")

    mask = df['Price_Billion'].isna() & df['Price_Raw'].notna()
    df.loc[mask, 'Price_Billion'] = df.loc[mask, 'Price_Raw'].apply(parse_price_raw)

    mask = df['Price_Raw'].isna() & df['Price_Billion'].notna()
    df.loc[mask, 'Price_Raw'] = df.loc[mask, 'Price_Billion'] \
        .apply(lambda x: f"{x:.2f}".replace('.', ',') + " tỷ")

    before = len(df)
    df = df.dropna(subset=['Price_Raw', 'Price_Billion'], how='all')
    print(f"Dropped rows (missing both prices): {before - len(df)}")

    return df


Hàm extract diện tích từ text

In [8]:
def extract_area_from_text(text):
    if pd.isna(text):
        return np.nan
    text = str(text).lower()
    match = re.search(r'(\d{2,4})\s*(m2|m²)', text)
    if match:
        return float(match.group(1))
    return np.nan


In [10]:
def clean_area(df):
    print("===== AREA CLEANING =====")

    mask = df['Area_m2'].isna() & df['Title'].notna()
    df.loc[mask, 'Area_m2'] = df.loc[mask, 'Title'].apply(extract_area_from_text)

    mask = df['Area_m2'].isna() & df['Description'].notna()
    df.loc[mask, 'Area_m2'] = df.loc[mask, 'Description'].apply(extract_area_from_text)

    before = len(df)
    df = df.dropna(subset=['Area_m2'])
    print(f"Dropped rows (missing Area): {before - len(df)}")

    df = df[(df['Area_m2'] > 10) & (df['Area_m2'] < 1000)]
    return df


Kiểm tra tình trạng thiếu Bedrooms & Toilets

Hàm extract Bedrooms từ text

In [11]:
def extract_bedrooms_from_text(text):
    if pd.isna(text):
        return np.nan
    text = str(text).lower()

    patterns = [
        r'(\d+)\s*(phòng ngủ|phong ngu)',
        r'(\d+)\s*pn\b',
        r'pn\s*[:\-]?\s*(\d+)',
        r'(\d+)\s*(bedroom|br)\b'
    ]

    for p in patterns:
        m = re.search(p, text)
        if m:
            return float(m.group(1))
    return np.nan


In [12]:
def clean_bedrooms(df):
    print("===== BEDROOM CLEANING =====")
    df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')

    mask = df['Bedrooms'].isna() & df['Title'].notna()
    df.loc[mask, 'Bedrooms'] = df.loc[mask, 'Title'].apply(extract_bedrooms_from_text)

    mask = df['Bedrooms'].isna() & df['Description'].notna()
    df.loc[mask, 'Bedrooms'] = df.loc[mask, 'Description'].apply(extract_bedrooms_from_text)

    return df


Hàm extract Toilets từ text

In [13]:
def extract_toilets_from_text(text):
    if pd.isna(text):
        return np.nan
    text = str(text).lower()

    patterns = [
        r'(\d+)\s*(wc|toilet|toilets)',
        r'(\d+)\s*(nhà vệ sinh|nha ve sinh)',
        r'(\d+)\s*(phòng vệ sinh|phong ve sinh)',
        r'wc\s*[:\-]?\s*(\d+)',
        r'vs\s*[:\-]?\s*(\d+)'
    ]

    for p in patterns:
        m = re.search(p, text)
        if m:
            return float(m.group(1))
    return np.nan


In [14]:
def clean_toilets(df):
    print("===== TOILETS CLEANING =====")
    df['Toilets'] = pd.to_numeric(df['Toilets'], errors='coerce')

    mask = df['Toilets'].isna() & df['Title'].notna()
    df.loc[mask, 'Toilets'] = df.loc[mask, 'Title'].apply(extract_toilets_from_text)

    mask = df['Toilets'].isna() & df['Description'].notna()
    df.loc[mask, 'Toilets'] = df.loc[mask, 'Description'].apply(extract_toilets_from_text)

    return df


In [15]:
def check_bedroom_toilet_missing(df):
    b = df['Bedrooms'].isna()
    t = df['Toilets'].isna()

    print("===== BEDROOM / TOILET STATUS =====")
    print("Missing BOTH     :", (b & t).sum())
    print("Missing ONLY ONE :", (b ^ t).sum())
    print("Complete BOTH    :", (~b & ~t).sum())


Drop row thiếu Bedrooms HOẶC Toilets

In [16]:
def drop_missing_bed_toilet(df):
    print("===== DROP MISSING BEDROOM / TOILET =====")
    before = len(df)
    df = df[df['Bedrooms'].notna() & df['Toilets'].notna()].copy()
    print(f"Total BEFORE : {before}")
    print(f"Total AFTER  : {len(df)}")
    print(f"Dropped      : {before - len(df)}")
    return df


In [17]:
df = df_raw.copy()

df = clean_price(df)
df = clean_area(df)
df = clean_bedrooms(df)
df = clean_toilets(df)

report_missing_values(df)
check_bedroom_toilet_missing(df)

df_ready = drop_missing_bed_toilet(df)


===== PRICE CLEANING =====
Dropped rows (missing both prices): 25
===== AREA CLEANING =====
Dropped rows (missing Area): 0
===== BEDROOM CLEANING =====
===== TOILETS CLEANING =====
===== MISSING VALUE REPORT =====
               Missing_Count  Percent_Missing (%)
Toilets                  672                47.12
Bedrooms                 559                39.20
Title                      0                 0.00
Page                       0                 0.00
id                         0                 0.00
Price_Billion              0                 0.00
Price_Raw                  0                 0.00
District                   0                 0.00
Area_m2                    0                 0.00
Address                    0                 0.00
Post_Time                  0                 0.00
Link                       0                 0.00
Description                0                 0.00
===== BEDROOM / TOILET STATUS =====
Missing BOTH     : 554
Missing ONLY ONE : 123
Comp

D:\Temp\ipykernel_22472\699342373.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'Price_Billion'] = df.loc[mask, 'Price_Raw'].apply(parse_price_raw)


DATASET CUỐI

In [18]:
print("FINAL ROWS:", len(df_ready))
df_ready.head()


FINAL ROWS: 749


,id,Page,Title,Price_Raw,Price_Billion,Area_m2,District,Address,Bedrooms,Toilets,Post_Time,Link,Description
0,303270749,1,"Chi 4.68 Tỷ, 2 tầng 62m2. Mặt tiền kinh doanh,...","4,68 tỷ",4.68,62.0,Huyện Nhà Bè,"H. Nhà Bè, Tp. Hồ Chí Minh",2.0,2.0,17/12/2025,https://meeyland.com/ban-nha-rieng-nha-be-ho-c...,"Nhà cấp 4, 2 tầng, 62m2. Mặt tiền kinh doanh, ..."
3,306057087,1,"Nhà 5 tầng kiên cố, 52m2, hẻm nhựa 8m, Vườn Là...",6 tỷ,6.0,52.0,Huyện Hêm,"P. Thạnh Lộc, Q. 12, Tp. Hồ Chí Minh",5.0,5.0,17/12/2025,https://meeyland.com/ban-nha-rieng-quan-12-ho-...,"Bước vào căn nhà này, bạn sẽ cảm nhận được sự ..."
4,306216130,1,Chỉ 2 tỷ sở hữu ngay shophoue khối đế vừa kinh...,"2,1 tỷ",2.1,58.0,Huyện Hêm,"Đ. Nguyễn Văn Linh, P. Tân Thuận Tây, Q. 7, Tp...",1.0,1.0,17/12/2025,https://meeyland.com/ban-can-ho-chung-cu-quan-...,Mở bán SHOPHOUSE khối đế dự án Acsent Lakeside...
5,305875815,1,[P BÀN CỜ] BÁN NHÀ MỚI ĐẸP 5 TẦNG Ở NGAY HXH P...,"9,9 tỷ",9.9,29.0,Huyện Hêm,"Đ. Nguyễn Đình Chiểu, P. 2, Q. 3, Tp. Hồ Chí Minh",3.0,3.0,17/12/2025,https://meeyland.com/ban-nha-rieng-quan-3-ho-c...,"Nhà mới xây, dọn vào ở ngay, không cần sửa.\nT..."
6,306068101,1,"Chào mừng quý khách đến với ""căn nhà trong mơ""...","6,25 tỷ",6.25,41.4,Huyện Hêm,"Đ. Xô Viết Nghệ Tĩnh, P. 21, Q. Bình Thạnh, Tp...",3.0,3.0,17/12/2025,https://meeyland.com/ban-nha-rieng-binh-thanh-...,"Chào mừng quý khách đến với ""căn nhà trong mơ""..."


Cleaning Address

In [25]:
def extract_district_final(address):
    if pd.isna(address):
        return np.nan

    # 1. Tiền xử lý: Xóa "xem thêm", đưa về chữ thường, nén nhiều khoảng trắng thành 1
    text = str(address).lower()
    text = text.replace("xem thêm", "")
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. Xử lý đặc biệt cho Thủ Đức
    if 'thủ đức' in text:
        return 'Thành phố Thủ Đức'

    # 3. Dùng Regex để bắt Quận/Q + Số (Chấp nhận có hoặc không có khoảng trắng ở giữa)
    # \s* đại diện cho việc có 0 hoặc nhiều khoảng trắng
    # ([0-9]{1,2}) bắt 1 hoặc 2 chữ số
    match_num = re.search(r'(?:quận|q)\s*([0-9]{1,2})', text)
    if match_num:
        num = int(match_num.group(1)) # Ép kiểu int để "01" thành "1"
        return f"Quận {num}"

    # 4. Nếu không phải quận số, kiểm tra danh sách Quận tên chữ (Whitelist)
    # Cách này an toàn nhất để tránh "Huyện Hêm", "Huyện Thạnh"
    districts_vietnamese = {
        'bình thạnh': 'Quận Bình Thạnh',
        'gò vấp': 'Quận Gò Vấp',
        'tân bình': 'Quận Tân Bình',
        'tân phú': 'Quận Tân Phú',
        'bình tân': 'Quận Bình Tân',
        'phú nhuận': 'Quận Phú Nhuận',
        'bình chánh': 'Huyện Bình Chánh',
        'hóc môn': 'Huyện Hóc Môn',
        'củ chi': 'Huyện Củ Chi',
        'nhà bè': 'Huyện Nhà Bè',
        'cần giờ': 'Huyện Cần Giờ'
    }

    for key, value in districts_vietnamese.items():
        if key in text:
            return value

    return np.nan

In [26]:
# Cập nhật lại những dòng đang bị sai hoặc thiếu
df_ready['District'] = df_ready['Address'].apply(extract_district_final)

# Kiểm tra lại 10 mẫu ngẫu nhiên
print(df_ready[['District', 'Address']].sample(10))

# Xem danh sách các Quận đã được gom nhóm
print(df_ready['District'].unique())

             District                                            Address
1190              NaN                              Q. 7, Tp. Hồ Chí Minh
877               NaN                              Q. 7, Tp. Hồ Chí Minh
168               NaN       15, Đ. 19, P. Tân Quy, Q. 7, Tp. Hồ Chí Minh
1266              NaN                                           Xem thêm
1223              NaN                                           Xem thêm
128               NaN                                           Xem thêm
1350              NaN                P. Tân Phong, Q. 7, Tp. Hồ Chí Minh
644               NaN  Đ. Nguyễn Chí Thanh, P. 8, Q. 10, Tp. Hồ Chí Minh
1167  Quận Bình Thạnh  Đ. Xô Viết Nghệ Tĩnh, P. 25, Q. Bình Thạnh, Tp...
692   Quận Bình Thạnh  Đ. Đặng Thùy Trâm, P. 13, Q. Bình Thạnh, Tp. H...
['Huyện Nhà Bè' nan 'Quận Bình Thạnh' 'Thành phố Thủ Đức' 'Quận Tân Phú'
 'Quận Gò Vấp' 'Quận Tân Bình' 'Quận Phú Nhuận' 'Huyện Bình Chánh'
 'Quận Bình Tân' 'Huyện Hóc Môn' 'Huyện Củ Chi']


In [27]:

def extract_district_from_address(address):
    if pd.isna(address):
        return np.nan

    # 1. TIỀN XỬ LÝ: Đưa về chữ thường, xóa "xem thêm", nén khoảng trắng
    text = str(address).lower()
    text = text.replace("xem thêm", "")
    # Xóa các dấu chấm sau các từ viết tắt để dễ xử lý (q. -> q, h. -> h)
    text = re.sub(r'(?<=\b[qht])\.', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. ƯU TIÊN CAO NHẤT: Thành phố Thủ Đức
    # (Bao gồm các kiểu: tp thủ đức, thành phố thủ đức, q thủ đức...)
    if any(kw in text for kw in ['thủ đức', 'thu duc']):
        return 'Thành phố Thủ Đức'

    # 3. XỬ LÝ QUẬN SỐ (Q1, Q 1, Q 01, Quận 1, Quận 01, Q.1...)
    # Giải thích Regex: (?:quận|q) tìm 'quận' hoặc 'q'. \s* cho phép nhiều khoảng trắng.
    # ([0-9]{1,2}) bắt lấy 1 hoặc 2 chữ số.
    match_num = re.search(r'\b(?:quận|q)\s*([0-9]{1,2})\b', text)
    if match_num:
        num = int(match_num.group(1)) # Ép kiểu int để biến 01, 02 thành 1, 2
        return f"Quận {num}"

    # 4. XỬ LÝ QUẬN/HUYỆN TÊN CHỮ (Bình Thạnh, Gò Vấp, Nhà Bè...)
    # Dùng Whitelist để tránh bắt nhầm "Huyện Hêm" từ "Xem thêm"
    dist_map = {
        'bình thạnh': 'Quận Bình Thạnh',
        'gò vấp': 'Quận Gò Vấp',
        'tân bình': 'Quận Tân Bình',
        'tân phú': 'Quận Tân Phú',
        'bình tân': 'Quận Bình Tân',
        'phú nhuận': 'Quận Phú Nhuận',
        'bình chánh': 'Huyện Bình Chánh',
        'hóc môn': 'Huyện Hóc Môn',
        'củ chi': 'Huyện Củ Chi',
        'nhà bè': 'Huyện Nhà Bè',
        'cần giờ': 'Huyện Cần Giờ',
        'quận 12': 'Quận 12' # Đề phòng trường hợp quận số viết chữ
    }

    for key, value in dist_map.items():
        if key in text:
            return value

    # 5. CÁC QUẬN CÒN LẠI (Quận 1 đến Quận 11 - nếu viết kiểu chữ)
    for i in range(1, 12):
        if f'quận {i}' in text or f'q {i}' in text:
            return f"Quận {i}"

    return np.nan

def clean_project_district(df):
    print("===== BẮT ĐẦU CHUYỂN ĐỔI ADDRESS -> DISTRICT =====")

    # Tạo mask: Những dòng District đang thiếu NHƯNG Address có dữ liệu
    mask = df['District'].isna() & df['Address'].notna()
    print(f"Số dòng cần xử lý: {mask.sum()}")

    # Thực hiện chuyển đổi
    extracted_data = df.loc[mask, 'Address'].apply(extract_district_from_address)

    # Điền vào cột District
    df.loc[mask, 'District'] = extracted_data

    print("Kết quả sau khi xử lý:")
    print(df['District'].value_counts())
    print(f"Số dòng vẫn còn thiếu District: {df['District'].isna().sum()}")

    return df

# --- THỰC THI ---
# Giả sử df_ready là dataframe của bạn
df_ready = clean_project_district(df_ready)

# Kiểm tra các mẫu dữ liệu thực tế bạn đã đưa ra
test_indices = [1190, 877, 168, 1350, 644, 1167]
# Lọc những index có tồn tại trong df_ready để in ra kiểm tra
existing_indices = [i for i in test_indices if i in df_ready.index]
print("\nKIỂM TRA MẪU SAU KHI FIX:")
print(df_ready.loc[existing_indices, ['District', 'Address']])

===== BẮT ĐẦU CHUYỂN ĐỔI ADDRESS -> DISTRICT =====
Số dòng cần xử lý: 333
Kết quả sau khi xử lý:
District
Quận Tân Bình        88
Quận 7               75
Quận Bình Thạnh      68
Quận Tân Phú         62
Quận Gò Vấp          57
Thành phố Thủ Đức    47
Quận 10              34
Quận 12              32
Quận Phú Nhuận       25
Quận Bình Tân        24
Huyện Hóc Môn        24
Quận 1               22
Quận 3               19
Huyện Bình Chánh     14
Quận 6                9
Quận 8                9
Quận 5                7
Huyện Nhà Bè          6
Quận 11               4
Quận 4                2
Huyện Củ Chi          1
Name: count, dtype: int64
Số dòng vẫn còn thiếu District: 120

KIỂM TRA MẪU SAU KHI FIX:
             District                                            Address
1190           Quận 7                              Q. 7, Tp. Hồ Chí Minh
877            Quận 7                              Q. 7, Tp. Hồ Chí Minh
168            Quận 7       15, Đ. 19, P. Tân Quy, Q. 7, Tp. Hồ Chí Minh
1350     

In [29]:
# 1. Lấy danh sách các dòng đã được chỉnh sửa (District từ NaN thành có giá trị)
# Giả sử mask_processed là những dòng mà trước đó District bị thiếu
df_fixed = df_ready[df_ready['Address'].notna() & df_ready['District'].notna()].copy()

# 2. Lấy danh sách các dòng vẫn chưa thể thay đổi (Vẫn là NaN)
df_remain_nan = df_ready[df_ready['District'].isna()].copy()

print("--- THỐNG KÊ KẾT QUẢ ---")
print(f"Số dòng đã sửa đúng: {len(df_fixed)}")
print(f"Số dòng vẫn còn trống: {len(df_remain_nan)}")

# 3. Hiển thị bảng so sánh các dòng đã sửa (Top 10 dòng đầu)
print("\n===== BẢNG 1: CÁC DÒNG ĐÃ CHỈNH SỬA ĐÚNG (MẪU) =====")
if not df_fixed.empty:
    print(df_fixed[['Address', 'District']].head(10).to_markdown())
else:
    print("Không có dòng nào được chỉnh sửa.")

# 4. Hiển thị bảng các dòng chưa sửa được (Để kiểm tra lý do)
print("\n===== BẢNG 2: CÁC DÒNG CHƯA THỂ THAY ĐỔI (CẦN KIỂM TRA) =====")
if not df_remain_nan.empty:
    # Chỉ lấy cột Address để soi lỗi
    print(df_remain_nan[['Address']].head(10).to_markdown())
else:
    print("Tuyệt vời! Không còn dòng nào bị thiếu.")

--- THỐNG KÊ KẾT QUẢ ---
Số dòng đã sửa đúng: 629
Số dòng vẫn còn trống: 120

===== BẢNG 1: CÁC DÒNG ĐÃ CHỈNH SỬA ĐÚNG (MẪU) =====
|    | Address                                                      | District          |
|---:|:-------------------------------------------------------------|:------------------|
|  0 | H. Nhà Bè, Tp. Hồ Chí Minh                                   | Huyện Nhà Bè      |
|  3 | P. Thạnh Lộc, Q. 12, Tp. Hồ Chí Minh                         | Quận 12           |
|  4 | Đ. Nguyễn Văn Linh, P. Tân Thuận Tây, Q. 7, Tp. Hồ Chí Minh  | Quận 7            |
|  5 | Đ. Nguyễn Đình Chiểu, P. 2, Q. 3, Tp. Hồ Chí Minh            | Quận 3            |
|  6 | Đ. Xô Viết Nghệ Tĩnh, P. 21, Q. Bình Thạnh, Tp. Hồ Chí Minh  | Quận Bình Thạnh   |
| 10 | 284 Lê văn sỹ, Đ. Lê Văn Sỹ, P. 13, Q. 3, Tp. Hồ Chí Minh    | Quận 3            |
| 12 | P. Long Trường, Tp. Thủ Đức, Tp. Hồ Chí Minh                 | Thành phố Thủ Đức |
| 13 | Đ. Điện Biên Phủ, P. 25, Q. Bình Thạnh, Tp. Hồ Chí M

In [32]:
# 'utf-8-sig' giúp Excel nhận diện đúng tiếng Việt có dấu
df_fixed.to_csv("../data/cleaned/cleanMey.csv", index=False, encoding='utf-8-sig')